In [11]:
import pandas as pd
import scanpy as sc

from tasks.data_handling import data_pre_processing as dpp

### Read list of signature genes from txt and pick out suva classes

In [31]:
file1 = open('NOsigdb_may2023.txt', 'r')
Lines = file1.readlines()
df_resp = None
for line in Lines:
    strip = line.strip().split('\t')
    df_temp = pd.DataFrame(data={'genes': strip[2:]})
    df_temp.loc[:, 'set'] = [strip[0]] * df_temp.shape[0]
    df_resp = pd.concat([df_resp, df_temp])
df_suva = df_resp[df_resp['set'].str.startswith('suva')]
suva_categories = df_suva['set'].unique()

In [8]:
print(df_suva)
print(suva_categories)

      genes                     set
0      CST3   suva_ac_PMID_31327527
1     S100B   suva_ac_PMID_31327527
2    SLC1A3   suva_ac_PMID_31327527
3     HEPN1   suva_ac_PMID_31327527
4      HOPX   suva_ac_PMID_31327527
..      ...                     ...
45    FABP5  suva_opc_PMID_31327527
46    NLGN3  suva_opc_PMID_31327527
47  SERINC5  suva_opc_PMID_31327527
48  EPB41L2  suva_opc_PMID_31327527
49  GPR37L1  suva_opc_PMID_31327527

[363 rows x 2 columns]
['suva_ac_PMID_31327527' 'suva_g1s_PMID_31327527' 'suva_g2m_PMID_31327527'
 'suva_mes1_PMID_31327527' 'suva_mes2_PMID_31327527'
 'suva_npc1_PMID_31327527' 'suva_npc2_PMID_31327527'
 'suva_opc_PMID_31327527']


### Pre-process data

In [32]:
file_name = '3013_P_filtered_feature_bc_matrix.h5'
data_path = f'../example/data/{file_name}'
adata = sc.read_10x_h5(data_path)
adata.var_names_make_unique()
dpp.quality_control(adata=adata)
adata_cells_removed = dpp.remove_bad_cells(adata=adata,
                                           max_n_genes=10000, 
                                           min_n_genes=500,
                                           mitochondrial_percent=20)
dpp.normalize_data(adata=adata_cells_removed)
adata_pre_processed = dpp.filter_genes(adata=adata_cells_removed, 
                                        min_n_cells=50)

adata_for_function = adata_pre_processed.copy()


C:\Users\ronja\anaconda3\envs\transformer_bio\lib\site-packages\anndata\_core\anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\ronja\anaconda3\envs\transformer_bio\lib\site-packages\anndata\_core\anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\ronja\anaconda3\envs\transformer_bio\lib\site-packages\scanpy\preprocessing\_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
C:\Users\ronja\anaconda3\envs\transformer_bio\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:539: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['hvg'] = {'flavor': flavor}


### Add class scores to each cell

In [33]:
for suva_class in suva_categories:
    gene_list = df_suva.loc[df_suva['set'] == suva_class, 'genes'].tolist()
    sc.tl.score_genes(adata_pre_processed, score_name=suva_class, gene_list=gene_list)
    sc.tl.score_genes(adata_cells_removed, score_name=suva_class, gene_list=gene_list)
# See below that not all genes are in the set, even without filtering out genes. What does that mean?

#### Note that some cells below have negative values in some classes. Should we go for absolute values?

In [34]:
adata_pre_processed.obs[suva_categories.tolist()]

,suva_ac_PMID_31327527,suva_g1s_PMID_31327527,suva_g2m_PMID_31327527,suva_mes1_PMID_31327527,suva_mes2_PMID_31327527,suva_npc1_PMID_31327527,suva_npc2_PMID_31327527,suva_opc_PMID_31327527
AAACCCACAAGCGAAC-1,0.495610,0.392755,-0.108044,0.493775,0.299444,0.082490,0.095204,0.198630
AAACCCACAATTTCTC-1,0.450129,-0.154239,-0.105096,0.178716,0.057103,0.100310,0.124584,0.366578
AAACCCACACCAAATC-1,0.547855,-0.099321,-0.154478,0.388681,0.298187,0.157651,0.115914,0.175814
AAACCCACACCGAATT-1,0.203442,0.057941,-0.159538,0.242374,0.193918,0.230980,0.196277,0.078090
AAACCCATCTTGGATG-1,0.678727,-0.018926,-0.176075,0.407065,0.201304,0.114754,0.091992,0.280367
...,...,...,...,...,...,...,...,...
TTTGGTTGTTTGTTCT-1,0.384686,0.305136,0.285282,0.555193,0.224498,0.098533,0.071534,0.091470
TTTGGTTTCCCAGGCA-1,0.168643,-0.005736,1.205045,0.624949,0.205460,0.195685,0.262083,-0.002447
TTTGGTTTCGTAGCCG-1,0.608639,-0.221996,-0.208454,0.568790,0.192038,0.103787,0.065481,0.154771
TTTGTTGTCAACCTCC-1,0.444351,0.704713,0.356173,0.392078,0.245069,0.041619,0.035515,0.154850


In [35]:
# Find the column with the maximum value for each row
max_class_column = adata_pre_processed.obs[suva_categories.tolist()].idxmax(axis=1)

# Annotate the result in a new column 'suva_class'
adata_pre_processed.obs['suva_class'] = max_class_column

print(adata_pre_processed.obs.suva_class)

AAACCCACAAGCGAAC-1      suva_ac_PMID_31327527
AAACCCACAATTTCTC-1      suva_ac_PMID_31327527
AAACCCACACCAAATC-1      suva_ac_PMID_31327527
AAACCCACACCGAATT-1    suva_mes1_PMID_31327527
AAACCCATCTTGGATG-1      suva_ac_PMID_31327527
                               ...           
TTTGGTTGTTTGTTCT-1    suva_mes1_PMID_31327527
TTTGGTTTCCCAGGCA-1     suva_g2m_PMID_31327527
TTTGGTTTCGTAGCCG-1      suva_ac_PMID_31327527
TTTGTTGTCAACCTCC-1     suva_g1s_PMID_31327527
TTTGTTGTCCCAAGCG-1    suva_npc1_PMID_31327527
Name: suva_class, Length: 1432, dtype: object


### Lastly save the h5-file

In [30]:
# Specify the file path where you want to save the h5 file
output_file_path = f'classified_{file_name}'

# Save the AnnData object to an h5 file
adata_pre_processed.write_h5ad(output_file_path)

## Test that written function runs

In [37]:
from cell_classification import classify_cells

temp_adata = classify_cells(pre_processed_adata=adata_for_function,
                            class_df=df_suva)
print(temp_adata.obs.suva_class)
print(adata_pre_processed.obs.suva_class)

AAACCCACAAGCGAAC-1      suva_ac_PMID_31327527
AAACCCACAATTTCTC-1      suva_ac_PMID_31327527
AAACCCACACCAAATC-1      suva_ac_PMID_31327527
AAACCCACACCGAATT-1    suva_mes1_PMID_31327527
AAACCCATCTTGGATG-1      suva_ac_PMID_31327527
                               ...           
TTTGGTTGTTTGTTCT-1    suva_mes1_PMID_31327527
TTTGGTTTCCCAGGCA-1     suva_g2m_PMID_31327527
TTTGGTTTCGTAGCCG-1      suva_ac_PMID_31327527
TTTGTTGTCAACCTCC-1     suva_g1s_PMID_31327527
TTTGTTGTCCCAAGCG-1    suva_npc1_PMID_31327527
Name: suva_class, Length: 1432, dtype: category
Categories (8, object): ['suva_ac_PMID_31327527', 'suva_g1s_PMID_31327527', 'suva_g2m_PMID_31327527', 'suva_mes1_PMID_31327527', 'suva_mes2_PMID_31327527', 'suva_npc1_PMID_31327527', 'suva_npc2_PMID_31327527', 'suva_opc_PMID_31327527']
AAACCCACAAGCGAAC-1      suva_ac_PMID_31327527
AAACCCACAATTTCTC-1      suva_ac_PMID_31327527
AAACCCACACCAAATC-1      suva_ac_PMID_31327527
AAACCCACACCGAATT-1    suva_mes1_PMID_31327527
AAACCCATCTTGGATG-1      su